### RW ML Kits

https://github.com/ravindutw

© Ravindu Wijesundara

# PyTorch Deep Learning Template

In [ ]:
#import warnings
#warnings.filterwarnings("ignore")

In [ ]:
# Run following on Google Colab
#!pip install s3fs

In [ ]:
import pandas as pd

## Import data

In [ ]:
data = pd.read_parquet("DATA_LOCATION")
#data = pd.read_csv("DATA_LOCATION")

In [ ]:
pd.set_option('display.max_columns', None)
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

## Preprocessing

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import make_column_selector
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import TargetEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
X = df.drop(columns=['Traget_column']) # Replace with target column
y = df['Traget_column']

In [ ]:
class_names = y.unique().tolist()

In [ ]:
column_names = X.columns.tolist() # List all column names
numerical_cols = X.select_dtypes(include='number').columns.tolist() # List all numerical column names
cat_cols = X.select_dtypes(include='object').columns.tolist() # List all categorical column names

In [ ]:
drop_features = [] # List features to drop

ordinal_encode_column_features = [] # List features to ordinal encode

# List target columns or automatically select all the remaining categorical columns
target_encode_features = [c for c in cat_cols if c not in ordinal_encode_column_features]
#target_encode_features = []

# Automatically select features to one-hot encode
oh_encode_features = [c for c in cat_cols if c not in (ordinal_encode_column_features + target_encode_features)]
#oh_encode_features = []

pass_through_features = [] # Columns to pass-through (If any)

In [ ]:
ordinal_encoder = OrdinalEncoder()
ordinal_encode_column_names = [c for c in ordinal_encode_column_features if c not in drop_features]

In [ ]:
target_encoder = TargetEncoder(target_type='binary', cv=5, smooth='auto', random_state=42) # Change params accordingly
target_encode_column_names = [c for c in target_encode_features if c not in drop_features]

In [ ]:
oh_encoder = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='ignore') # Change params accordingly
oh_encode_column_names = [c for c in oh_encode_features if c not in drop_features]

In [ ]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
columns_to_scale = [c for c in features_to_scale if c not in drop_features] # Scale all numerical columns

#### Preprocessor

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal_enc', ordinal_encoder, ordinal_encode_column_names),
        ('target_enc', target_encoder, target_encode_column_names),
        ('oh_enc', oh_encoder, oh_encode_column_names),
        ('min_max_scale', min_max_scaler, columns_to_scale),
        ('passthrough', 'passthrough', pass_through_features)
    ],
    remainder='drop', # Drop everything that are not in the transformer
    verbose_feature_names_out=True
)
# Comment-out unncecessary transformations

In [ ]:
# Encode y
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

## Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Preprocessing

In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [ ]:
X_train_processed = pipeline.fit_transform(X_train, y_train)
X_test_processed = pipeline.transform(X_test)

#### Integrate with torch

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class Dataset:
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
train_dataset = Dataset(X_train_processed, y_train)
test_dataset = Dataset(X_test_processed, y_test)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # Update batch size accordingly
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

## Model

In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
input_size = X_train_processed.shape[1]

In [ ]:
# A sample neural network

class ClassifierModel(nn.Module):
    
    def __init__(self, input_size):
        super(ClassifierModel, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(64, 128)
        self.relu = nn.ReLU()
        self.layer3 = nn.Linear(128, 128)
        self.relu = nn.ReLU()
        self.layer4 = nn.Linear(128, 256)
        self.relu = nn.ReLU()
        self.layer5 = nn.Linear(256, 256)
        self.relu = nn.ReLU()
        self.layer6 = nn.Linear(256, 128)
        self.relu = nn.ReLU()
        self.layer7 = nn.Linear(128, 64)
        self.output_layer = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.relu(self.layer4(x))
        x = self.relu(self.layer5(x))
        x = self.relu(self.layer6(x))
        x = self.relu(self.layer7(x))
        x = self.sigmoid(self.output_layer(x))
        return x

In [ ]:
classifier_model = ClassifierModel(input_size)

In [ ]:
# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(classifier_model. parameters(), lr=0.001)

## Train

In [ ]:
epochs = 15 # No. of epochs

In [ ]:
for epoch in range(epochs):
    for X_batch, y_batch in train_loader:
        outputs = classifier_model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

## Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def test_model(classifier_model, test_loader, device, class_names):
    classifier_model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = classifier_model(X_batch)
            predictions = (outputs > 0.5).int()
            all_preds.extend(predictions.numpy())
            all_labels.extend(y_batch.numpy())

    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    cm = confusion_matrix(all_labels, all_preds)

    print("Confusion Matrix:")
    print(cm)

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.plasma)
    plt.title("Confusion matrix")
    plt.show()

In [ ]:
test_model(classifier_model, test_loader, device, class_names)